In [1]:
import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
import scipy.optimize as optimize
import matplotlib.pyplot as plt
from IPython.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
class yield_and_spread_analysis:
    
    @staticmethod
    def next_coupon_date(settle_date, maturity_date, coupon_frequency):
        
        '''
        Return next coupon date.
        
        settle_date (str): settlement date to which the cashflows will be discounted "%Y-%m-%d";
        maturity_date (str): maturity of the bond, in format "%Y-%m-%d";
        coupon_frequency (int): coupon frequency.
        
        '''   
        
        if type(settle_date) is str and type(maturity_date) is str:

            settle_date = datetime.strptime(settle_date, "%Y-%m-%d")
            maturity_date = datetime.strptime(maturity_date, "%Y-%m-%d")
            
        interval_months = 12 // coupon_frequency
    
        coupon_dates = []
        current_date = maturity_date
        
        while current_date > settle_date:
            
            coupon_dates.append(current_date)
            year = current_date.year
            month = current_date.month - interval_months
            
            if month <= 0:
                
                year -= 1
                month += 12
                
            last_day_of_month = calendar.monthrange(year, month)[1]
            current_date = current_date.replace(year = year, month = month, day = min(current_date.day, last_day_of_month))
    
        for date in reversed(coupon_dates):
            
            if date > settle_date:
                
                return date.strftime("%Y-%m-%d")
    
        return None 
    
    @staticmethod
    def previous_coupon_date(settle_date, maturity_date, coupon_frequency):
        
        '''
        Return previous coupon date.
        
        settle_date (str): settlement date to which the cashflows will be discounted "%Y-%m-%d";
        maturity_date (str): maturity of the bond, in format "%Y-%m-%d"
        coupon_frequency (int): coupon frequency.
        
        '''   

        if type(settle_date) is str and type(maturity_date) is str:

            settle_date = datetime.strptime(settle_date, "%Y-%m-%d")
            maturity_date = datetime.strptime(maturity_date, "%Y-%m-%d")

        next_coupon_date = datetime.strptime(
            yield_and_spread_analysis.next_coupon_date(settle_date, maturity_date, coupon_frequency), "%Y-%m-%d"
        )

        interval_months = 12 // coupon_frequency

        year = next_coupon_date.year                                          
        month = next_coupon_date.month - interval_months   
    
        if month <= 0:
            year -= 1
            month += 12

        day = min(next_coupon_date.day, calendar.monthrange(year, month)[1])

        previous_coupon_date = datetime(year, month, day)

        return previous_coupon_date.strftime("%Y-%m-%d")
    
    @staticmethod
    def bond_accrued_interest(coupon, face_value, settle_date, maturity_date, coupon_frequency):
        
        '''
        Return accrued interest for a bullet bond with fixed coupon 
        
        coupon (float): annual coupon rate of the bond;
        face_value (int): face value of the bond;
        settle_date (str): settlement date to which the cashflows will be discounted "%Y-%m-%d";
        maturity_date (str): maturity of the bond, in format "%Y-%m-%d"
        coupon_frequency (int): coupon frequency.
        
        '''

        if type(settle_date) is str and type(maturity_date) is str:

            settle_date = datetime.strptime(settle_date, "%Y-%m-%d")
            maturity_date = datetime.strptime(maturity_date, "%Y-%m-%d")
        
        previous_coupon_date = datetime.strptime(yield_and_spread_analysis.previous_coupon_date(settle_date, 
                                                                                        maturity_date, 
                                                                                        coupon_frequency), 
                                             "%Y-%m-%d")
        
        next_coupon_date = datetime.strptime(yield_and_spread_analysis.next_coupon_date(settle_date, 
                                                                                        maturity_date, 
                                                                                        coupon_frequency), 
                                             "%Y-%m-%d")          
        
        accrued_interest = (coupon / coupon_frequency) * face_value * ((settle_date - previous_coupon_date).days / (next_coupon_date - previous_coupon_date).days)
        
        return accrued_interest
        
    
    @staticmethod
    def bond_dirty_price(ytm, coupon, face_value, settle_date, maturity_date, coupon_frequency):
        
        '''
        Return dirty price for a bullet bond with fixed coupon.
        
        ytm (float): annualised yield to maturity;
        coupon (float): annual coupon rate of the bond;
        face_value (int): face value of the bond;
        settle_date (str): settlement date to which the cashflows will be discounted "%Y-%m-%d";
        maturity_date (str): maturity of the bond, in format "%Y-%m-%d"
        coupon_frequency (int): coupon frequency.
        
        '''
        
        if type(settle_date) is str and type(maturity_date) is str:

            settle_date = datetime.strptime(settle_date, "%Y-%m-%d")
            maturity_date = datetime.strptime(maturity_date, "%Y-%m-%d")
        
        next_coupon_date = datetime.strptime(yield_and_spread_analysis.next_coupon_date(settle_date, 
                                                                                        maturity_date, 
                                                                                        coupon_frequency), 
                                             "%Y-%m-%d")      
        
        days_to_maturity = (maturity_date - settle_date).days
        T = (days_to_maturity / 365) * coupon_frequency         
        
        days_to_next_coupon = (next_coupon_date - settle_date).days
        v = (days_to_next_coupon / 365) * coupon_frequency
        
        N = T - v
                      
        price = 0
               
        for t in range(1, int(round(N)) + 1):
             
            price += coupon * face_value / coupon_frequency / (1 + ytm / coupon_frequency) ** t
        
        price += face_value / (1 + ytm / coupon_frequency) ** N
            
        return (price + (coupon * face_value / coupon_frequency)) / (1 + ytm / coupon_frequency) ** v
    
    @staticmethod
    def bond_clean_price(ytm, coupon, face_value, settle_date, maturity_date, coupon_frequency):

        '''
        Return clean price for a bullet bond with fixed coupon.
        
        ytm (float): annualised yield to maturity;
        coupon (float): annual coupon rate of the bond;
        face_value (int): face value of the bond;
        settle_date (str): settlement date to which the cashflows will be discounted "%Y-%m-%d";
        maturity_date (str): maturity of the bond, in format "%Y-%m-%d"
        coupon_frequency (int): coupon frequency.
        
        '''
        
        dirty_price = yield_and_spread_analysis.bond_dirty_price(ytm, coupon, face_value, settle_date, maturity_date, coupon_frequency)
        accrued_interest = yield_and_spread_analysis.bond_accrued_interest(coupon, face_value, settle_date, maturity_date, coupon_frequency)
    
        return dirty_price - accrued_interest
    
    @staticmethod
    def bond_ytm(price, coupon, face_value, settle_date, maturity_date, coupon_frequency):
        
        '''
        Return yield to maturity for a bullet bond with fixed coupon.
        
        price(float): bond dirty price;
        coupon(float): annual coupon rate of the bond;
        face_value(integer): face value of the bond;
        settle_date(string): settlement date to which the cashflows will be discounted "%Y-%m-%d";
        maturity_date(string): maturity of the bond, in format "%Y-%m-%d"
        coupon_frequency(integer): coupon frequency.
        
        '''
        
        if type(settle_date) is str and type(maturity_date) is str:

            settle_date = datetime.strptime(settle_date, "%Y-%m-%d")
            maturity_date = datetime.strptime(maturity_date, "%Y-%m-%d")
        
        days_to_maturity = (maturity_date - settle_date).days
        years_to_maturity = days_to_maturity / 365              
           
        def ytm_func(y):
            
            return yield_and_spread_analysis.bond_dirty_price(y, coupon, face_value, settle_date, maturity_date, coupon_frequency) - price
        
        ytm_initial_guess = 0.05
        
        ytm = optimize.newton(ytm_func, ytm_initial_guess)
        
        return ytm

    @staticmethod
    def bond_dv01(ytm, coupon, face_value, settle_date, maturity_date, coupon_frequency):

        '''
        Return DV01 for a bullet bond with fixed coupon.
        
        ytm: annualised yield to maturity, float;
        coupon: annual coupon rate of the bond, float;
        face_value: face value of the bond, integer;
        settle_date: settlement date to which the cashflows will be dicounted "%Y-%m-%d";
        maturity_date: maturity of the bond, in format "%Y-%m-%d"
        coupon_frequency: coupon frequency, integer.
        
        '''
        
        if type(settle_date) is str and type(maturity_date) is str:

            settle_date = datetime.strptime(settle_date, "%Y-%m-%d")
            maturity_date = datetime.strptime(maturity_date, "%Y-%m-%d")
        
        days_to_maturity = (maturity_date - settle_date).days
        years_to_maturity = days_to_maturity / 365
        
        price_up = yield_and_spread_analysis.bond_dirty_price(ytm - 0.01, coupon, face_value, settle_date, maturity_date, coupon_frequency)
        price_down = yield_and_spread_analysis.bond_dirty_price(ytm + 0.01, coupon, face_value, settle_date, maturity_date, coupon_frequency)
        
        return (price_up - price_down) / 2
    
    @staticmethod
    def bond_modified_duration(price, ytm, coupon, face_value, settle_date, maturity_date, coupon_frequency):
        
        '''
        Return Modified Duration for a bullet bond with fixed coupon.
        
        price: bond dirty price, float;
        ytm: annualised yield to maturity, float;
        coupon: annual coupon rate of the bond, float;
        face_value: face value of the bond, integer;
        settle_date: settlement date to which the cashflows will be dicounted "%Y-%m-%d";
        maturity_date: maturity of the bond, in format "%Y-%m-%d"
        coupon_frequency: coupon frequency, integer.
        
        '''
        
        bond_dv01 = yield_and_spread_analysis.bond_dv01(ytm, coupon, face_value, settle_date, maturity_date, coupon_frequency) 
        
        return bond_dv01 * 100 / price
    
    @staticmethod
    def bond_macaulay_duration(price, ytm, coupon, face_value, settle_date, maturity_date, coupon_frequency):
    
        '''
        Return Macaulay Duration for a bullet bond with fixed coupon.
        
        price: bond dirty price, float;
        ytm: annualised yield to maturity, float;
        coupon: annual coupon rate of the bond, float;
        face_value: face value of the bond, integer;
        settle_date: settlement date to which the cashflows will be dicounted "%Y-%m-%d";
        maturity_date: maturity of the bond, in format "%Y-%m-%d"
        coupon_frequency: coupon frequency, integer.
        
        '''
        
        mod_dur = yield_and_spread_analysis.bond_modified_duration(price, ytm, coupon, face_value, settle_date, maturity_date, coupon_frequency) 
        
        return mod_dur * (1 + ytm / coupon_frequency)
    
class yield_curve_analysis:
    
    '''Class of function to analyse and hedge yield curves.'''
    
    @staticmethod
    def forward_rates(spot_rates, maturities):
        
        '''
        Return forward rates based on input spot rates and maturities.
        
        spot_rates (list): spot rates;
        maturities(list): maturities.
        
        '''
              
        forward_rates = []
            
        forward_multiplier = 1
        
        for i, mat in enumerate(maturities):
            
            gross_forward_rate = ((1 + spot_rates[i]) ** mat) / forward_multiplier
                      
            forward_rates.append(gross_forward_rate - 1)
            
            forward_multiplier *= gross_forward_rate
        
        return forward_rates